In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Split training set into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# One-hot encode labels
encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_val = encoder.transform(y_val.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()


170498071/170498071 [==============================] - 11s 0us/step


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [3]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Evaluate model on test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_accuracy)

Epoch 1/10
1250/1250 [==============================] - 55s 43ms/step - loss: 1.8157 - accuracy: 0.3211 - val_loss: 1.4534 - val_accuracy: 0.4826
Epoch 2/10
1250/1250 [==============================] - 52s 41ms/step - loss: 1.4434 - accuracy: 0.4775 - val_loss: 1.2965 - val_accuracy: 0.5387
Epoch 3/10
1250/1250 [==============================] - 53s 42ms/step - loss: 1.3012 - accuracy: 0.5378 - val_loss: 1.1118 - val_accuracy: 0.6051
Epoch 4/10
1250/1250 [==============================] - 51s 41ms/step - loss: 1.2088 - accuracy: 0.5749 - val_loss: 1.1172 - val_accuracy: 0.5987
Epoch 5/10
1250/1250 [==============================] - 53s 42ms/step - loss: 1.1399 - accuracy: 0.6021 - val_loss: 1.0079 - val_accuracy: 0.6415
Epoch 6/10
1250/1250 [==============================] - 58s 46ms/step - loss: 1.0739 - accuracy: 0.6268 - val_loss: 0.9592 - val_accuracy: 0.6616
Epoch 7/10
1250/1250 [==============================] - 52s 42ms/step - loss: 1.0230 - accuracy: 0.6439 - val_loss: 0.9544 -

In [4]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json['image']
    image = np.array(data)
    image = image.reshape(1, 32, 32, 3)  # assuming input shape is (32, 32, 3)
    prediction = model.predict(image)
    class_idx = np.argmax(prediction)
    return jsonify({'class_index': int(class_idx)})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
